<a href="https://colab.research.google.com/github/GabHoo/GoTAutomyth_short/blob/main/generate_with_BART.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### start with installing libraries
!pip install transformers datasets 
!pip install rouge.score nltk py7zr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 7.0 MB/s 
     |████████████████████████████████| 451 kB 47.6 MB/s 
     |████████████████████████████████| 182 kB 51.7 MB/s 
     |████████████████████████████████| 7.6 MB 46.8 MB/s 
     |████████████████████████████████| 212 kB 9.1 MB/s 
     |████████████████████████████████| 115 kB 23.6 MB/s 
     |████████████████████████████████| 127 kB 49.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 65 kB 3.3 MB/s 
     |████████████████████████████████| 357 kB 20.6 MB/s 
     |████████████████████████████████| 379 kB 48.8 MB/s 
     |████████████████████████████████| 138 kB 46.3 MB/s 
     |███

In [ ]:
### import libraries

import transformers
from datasets import load_dataset, load_metric
from google.colab import drive
import numpy as np
import os
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
print(transformers.__version__)

4.24.0


In [ ]:
import torch

print(torch.__version__)

1.12.1+cu113


In [ ]:
### connect to the drive

MOUNTPOINT = '/content/gdrive'
Working_Dir = os.path.join(MOUNTPOINT, 'My Drive', 'Building_Narratives_Game_of_Thrones')
drive.mount(MOUNTPOINT)
print(Working_Dir)

Mounted at /content/gdrive
/content/gdrive/My Drive/Building_Narratives_Game_of_Thrones


In [ ]:
### upload the data

train_file = Working_Dir + '/data/train_data.json'
dev_file = Working_Dir + '/data/validation_data.json'
test_file = Working_Dir + '/data/test_data_event1.json'

dataset = load_dataset('json', data_files={'train': train_file, 'valid': dev_file, 'test': test_file})

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-a0d74b104867311f/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
### let's check the data

print(dataset['valid'][0]['story'][0][0])
print(dataset['valid'][0]['Knowledge Graph'][0])

Crossroads Inn was the place where the Robert Baratheon,King of House Greyjoyused to live. Robert Baratheon was also known around Crossroads Inn as a great warrior . It was the Summer Solstice when this story takes place.
the_Summer_Solstice - label - the Summer Solstice | Robert_Baratheon - type - Actor | Event - label - Event | hasOccupation - range - Occupation | warrior - type - Occupation | hasTitle - range - Title | Event_01 - type - OrdinaryWorld | warrior - label - warrior | King - type - Male | Event_01 - type - Event | House_Greyjoy - label - House Greyjoy | Crossroads_Inn - type - Place | Event - type - Class | Core - type - Class | King - type - Title | Event_01 - hasTime - the_Summer_Solstice | Event_01 - hasActor - Robert_Baratheon | hasPlace - range - Place | hasHouse - range - House | Event_01 - hasHouse - House_Greyjoy | Robert_Baratheon - type - Male | Core - label - Core | Event_01 - hasOccupation - warrior | Robert_Baratheon - label - Robert Baratheon | Event_01 - h

In [ ]:
### data looks OK, let's continue with the tokenizer
max_input = 512
max_target = 512
model_checkpoints = 'facebook/bart-base'
tokenizer = transformers.AutoTokenizer.from_pretrained(model_checkpoints)

Downloading:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
def preprocess_data(data_to_process):
  #get the dialogue text
  inputs = [graph[0] for graph in data_to_process['Knowledge Graph']]
  #tokenize text
  model_inputs = tokenizer(inputs,  max_length=max_input, padding='max_length', truncation=True)

  #tokenize labels
  #with tokenizer.as_target_tokenizer():
  targets = [target[0][0] for target in data_to_process['story']]
  model_targets = tokenizer(targets, max_length=max_target, padding='max_length', truncation=True)
    
  model_inputs['labels'] = model_targets['input_ids']
  #reuturns input_ids, attention_masks, labels
  return model_inputs

In [ ]:
tokenize_data = dataset.map(preprocess_data, batched = True) 

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenize_data

DatasetDict({
    train: Dataset({
        features: ['index', 'story', 'KG index', 'Knowledge Graph', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1365
    })
    valid: Dataset({
        features: ['index', 'story', 'KG index', 'Knowledge Graph', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 150
    })
    test: Dataset({
        features: ['index', 'story', 'KG index', 'Knowledge Graph', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 150
    })
})

In [ ]:
print(tokenize_data['test']['story'][3])

[['Once upon a time, in Riverlands there was a official whose name was Arya Stark. Arya Stark was a Queen  from the House Targaryen. It was sunrise when this story begins.']]


In [ ]:
print(tokenize_data['test']['Knowledge Graph'][3])

['official - label - official | Event_01 - hasOccupation - official | hasTime - range - Time | Event_01 - hasPlace - Riverlands | Core - label - Core | Riverlands - label - Riverlands | OrdinaryWorld - label - First stage of the story - Ordinary world presentation | Arya_Stark - label - Arya Stark | Event - type - Class | Arya_Stark - type - Main_Character | Arya_Stark - type - Actor | queen_consort - type - Title | House_Targaryen - label - House Targaryen | Event_01 - hasActor - Arya_Stark | Event_01 - label - Ordinary World presentation | Event_01 - hasTime - sunrise | hasPlace - range - Place | Arya_Stark - type - Female | Event_01 - journeyStage - 1 | official - type - Occupation | Event_01 - hasTitle - queen_consort | queen_consort - type - Actor | queen_consort - label - Queen  | House_Targaryen - type - House | hasOccupation - range - Occupation | Core - type - Class | Event - label - Event | queen_consort - type - Female | Event_01 - type - OrdinaryWorld | Event_01 - hasHouse 

In [ ]:
### let's drop the columns that we won't need
tokenize_data = dataset.map(preprocess_data, batched = True, remove_columns=['index', 'story', 'KG index', 'Knowledge Graph'])

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenize_data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1365
    })
    valid: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 150
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 150
    })
})

In [ ]:
### load model
model = transformers.AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)

Downloading:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [ ]:
batch_size = 4
#collator to create batches. It preprocess data with the given tokenizer
collator = transformers.DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
#####################
# metrics
# compute rouge for evaluation 
#####################
metric = load_metric('rouge')

def compute_rouge(pred):
  predictions, labels = pred
  #decode the predictions
  decode_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  #decode labels
  decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  #compute results
  res = metric.compute(predictions=decode_predictions, references=decode_labels, use_stemmer=True)
  #get %
  res = {key: value.mid.fmeasure * 100 for key, value in res.items()}

  pred_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  res['gen_len'] = np.mean(pred_lens)

  return {k: round(v, 4) for k, v in res.items()}

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  """


In [ ]:
args = transformers.Seq2SeqTrainingArguments(
    'output',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size= 1,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    generation_max_length = 128,
    generation_num_beams=5,
    eval_accumulation_steps=1,
    fp16=True
    )
#only CUDA available -> fp16=True

In [ ]:
### almost training time
trainer = transformers.Seq2SeqTrainer(
    model, 
    args,
    train_dataset=tokenize_data['train'],
    eval_dataset=tokenize_data['valid'],
    data_collator=collator,
    tokenizer=tokenizer,
    compute_metrics=compute_rouge
)

Using cuda_amp half precision backend


In [ ]:
### check GPU
!nvidia-smi

Tue Nov 15 09:06:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P0    27W /  70W |   1198MiB / 15109MiB |     11%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
### training time - takes around 15 mins

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1365
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 2
  Total optimization steps = 2046
  Number of trainable parameters = 139420416
You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,1.294600,0.002637,74.115200,60.605500,59.422000,59.289600,35.993300
1,0.003700,0.002534,74.236400,60.898400,59.520300,59.447800,35.813300
2,0.003000,0.002393,73.735200,60.020800,59.174500,59.004600,36.333300


Saving model checkpoint to output/checkpoint-500
Configuration saved in output/checkpoint-500/config.json
Model weights saved in output/checkpoint-500/pytorch_model.bin
tokenizer config file saved in output/checkpoint-500/tokenizer_config.json
Special tokens file saved in output/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 150
  Batch size = 1
Saving model checkpoint to output/checkpoint-1000
Configuration saved in output/checkpoint-1000/config.json
Model weights saved in output/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in output/checkpoint-1000/tokenizer_config.json
Special tokens file saved in output/checkpoint-1000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 150
  Batch size = 1
Saving model checkpoint to output/checkpoint-1500
Configuration saved in output/checkpoint-1500/config.json
Model weights saved in output/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in output/checkpoin

TrainOutput(global_step=2046, training_loss=0.3189716761383726, metrics={'train_runtime': 731.9213, 'train_samples_per_second': 5.595, 'train_steps_per_second': 2.795, 'total_flos': 1248130530017280.0, 'train_loss': 0.3189716761383726, 'epoch': 3.0})

In [ ]:
### save the model for later use

trainer.save_model(Working_Dir + "/tuned_models/BART_models/")

Saving model checkpoint to /content/gdrive/My Drive/Building_Narratives_Game_of_Thrones/tuned_models/BART_models/
Configuration saved in /content/gdrive/My Drive/Building_Narratives_Game_of_Thrones/tuned_models/BART_models/config.json
Model weights saved in /content/gdrive/My Drive/Building_Narratives_Game_of_Thrones/tuned_models/BART_models/pytorch_model.bin
tokenizer config file saved in /content/gdrive/My Drive/Building_Narratives_Game_of_Thrones/tuned_models/BART_models/tokenizer_config.json
Special tokens file saved in /content/gdrive/My Drive/Building_Narratives_Game_of_Thrones/tuned_models/BART_models/special_tokens_map.json


In [ ]:
### let's have a look at the predictions

preds, labels, metrics = trainer.predict(tokenize_data['test'], num_beams=5, min_length=50, max_length=128, no_repeat_ngram_size=2, early_stopping=True)

***** Running Prediction *****
  Num examples = 150
  Batch size = 1


In [ ]:
outfile_path = Working_Dir + '/generated_text/by_BART/generated_text_with_refs.txt'

outfile = open(outfile_path, "a", encoding='utf-8')

scores = metrics.items()
print(f'Results: {scores}')
outfile.write(f'Test Results: {scores}\n\n')

for gen, gold in zip(preds, labels):
  gen = tokenizer.decode(gen, skip_special_tokens=True)
  print(f'Generated text: {gen}')
  outfile.write(f'Generated text: {gen}\n')

  gold = tokenizer.decode(gold, skip_special_tokens=True)
  print(f'Reference text: {gold}')
  outfile.write(f'Reference text: {gold}\n\n')
  
outfile.close()

Results: dict_items([('test_loss', 0.0024106616619974375), ('test_rouge1', 66.4662), ('test_rouge2', 52.037), ('test_rougeL', 49.5821), ('test_rougeLsum', 49.6415), ('test_gen_len', 63.14), ('test_runtime', 109.4642), ('test_samples_per_second', 1.37), ('test_steps_per_second', 1.37)])
Generated text: It was the first day of Spring in Moat Cailin. Cersei Lannister was a  Queen  from the House Tyrell and worked as a military leader for a time. It was Easter day when this story takes place.
Reference text: Once upon a time, in Moat Cailin there was a military leader whose name was Cersei Lannister. Cersei Lannister was a Queen  from the House Tyrell. It was the first day of Spring when this story begins.
Generated text: It was the first day of Spring in Moat Cailin. Cersei Lannister was a  Queen  from the House Tyrell and worked as a military leader for a time. It was Easter day when this story takes place.
Reference text: It was the first day of Spring in Moat Cailin. Cersei Lannister w

In [ ]:
for i in metrics.items():
  print(i)

('test_loss', 0.0024106616619974375)
('test_rouge1', 66.4662)
('test_rouge2', 52.037)
('test_rougeL', 49.5821)
('test_rougeLsum', 49.6415)
('test_gen_len', 63.14)
('test_runtime', 109.4642)
('test_samples_per_second', 1.37)
('test_steps_per_second', 1.37)
